# コーヒーの生産の例

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/takazawa/PyOptBookColab/blob/main/examples/CoffeeProductionPlanning.ipynb)

## 1. そのまま実装

$$
\begin{array}{ll}
   \text{maximize}_{x_A, x_B \in \mathbb{R}} & 50x_A + 40x_B \\
   \text{subject to} & 0.5x_A \leq 80\\
		    & 0.5x_A + 0.7x_B \leq 180 \\
		    & 0.3x_B \leq 90 \\
		    & x_A \geq 0\\
		    & x_B \geq 0
\end{array}
$$

上記の問題をPyhonで表現するとどうなるだろうか？

In [ ]:
# Google Colabで実行する場合は初回に必ず実行する <- Pythonでは「#」からテキストを書くとコメント扱いになり無視される
import os
if 'COLAB_GPU' in os.environ:
    if not os.path.exists("PyOptBookColab"):
        !git clone https://github.com/takazawa/PyOptBookColab.git
        !cp PyOptBookColab/examples/*.csv .
        !pip install -r PyOptBookColab/requirements.txt -q

In [ ]:
# 必要なライブラリのインポート
import pulp

In [ ]:
# 問題の設定: どのような問題（最大化・最小化、線形計画等）であるかを定義する
problem = pulp.LpProblem("CoffeeProduction", pulp.LpMaximize)

In [ ]:
# 決定変数の設定
x_A = pulp.LpVariable('x_A', cat='Continuous')
x_B = pulp.LpVariable('x_B', cat='Continuous')

In [ ]:
# 目的関数の設定
problem += 50 * x_A + 40 * x_B

In [ ]:
# 制約条件の設定
problem += 0.5 * x_A <= 80
problem += 0.5 * x_A + 0.7 * x_B <= 180
problem += 0.3 * x_B <= 90
problem += x_A >= 0
problem += x_B >= 0

In [ ]:
# 表示
problem

In [ ]:
# 解を求める
status = problem.solve()
print('Status:', pulp.LpStatus[status])

In [ ]:
# 最適化結果の表示
print("x_A=", x_A.value(), "x_B=", x_B.value())

## リストを使って表記

コーヒー生産の問題は記号を作って次のようにもかける:

#### 集合（リスト）

- 豆の集合: $I = \{1, 2, 3\}$
- ブレンドコーヒーの集合: $J = \{A, B\}$

#### 定数

- ブレンドコーヒー$j \in J$ごとの1トン生産した時の利益: $p_j \geq 0$
  - $p_A=50$
  - $p_B=40$
- ブレンドコーヒー$j \in J$を1トン生産するために必要な豆$i \in I$の量: $a_{ij} \geq 0$
  - $a_{1, A}=0.5$
  -  $a_{2, A}=0.5$
  -  $a_{3, A}=0$
  -  $a_{1, B}=0$
  -  $a_{2, B}=0.7$
  -  $a_{3, B}=0.3$
-   豆 $j \in J$ごとの利用可能量: $b_j \geq 0$
  -  $b_1=80$
  -  $b_2=180$
  -  $b_3=90$

これらを使って最適化問題を定式化すると、


$$
\begin{array}{lll}
   \text{maximize}& \sum_{j \in J} p_jx_j &\\
   \text{subject to}&  \sum_{j \in J} a_{ij} x_j \leq b_i & i \in I,\\
   & x_j \geq 0 & j \in J.
\end{array}
$$

このような形式でPythonで書くとどうなるだろうか？

In [ ]:
# リストの定義
I = [1, 2, 3]
J = ["A", "B"]

# 定数
p = {"A": 50, "B": 40}
a = {(1, "A"): 0.5, (2, "A"): 0.5, (3, "A"): 0, (1, "B"): 0, (2, "B"): 0.7, (3, "B"): 0.3}
b = {1: 80, 2: 180, 3: 90}

In [ ]:
# 問題の設定: どのような問題（最大化・最小化、線形計画等）であるかを定義する
problem = pulp.LpProblem("CoffeeProduction", pulp.LpMaximize)

# 決定変数の設定
x = {}
for j in J:
    x[j] = pulp.LpVariable(f"x_{j}", cat="Continuous")

# 目的関数の設定
problem += sum([p[j] * x[j] for j in J])

# 制約条件の設定
for i in I:
    problem += sum([a[i, j] * x[j] for j in J]) <= b[i]
for j in J:
    problem += x[j] >= 0

problem

In [ ]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

In [ ]:
# 解の表示
for j in J:
    print(f"x_{j}={x[j].value()}")

# ファイルからデータを読み込む

In [ ]:
# pandasという表ファイルの処理を行うライブラリをインポートする
import pandas as pd

In [ ]:
df_p = pd.read_csv("coffee_p.csv")
df_b = pd.read_csv("coffee_b.csv")
df_a = pd.read_csv("coffee_a.csv")

In [ ]:
df_a

In [ ]:
df_b

In [ ]:
df_p

In [ ]:
J = list(df_p["coffee"])
I = list(df_b["bean"])
I, J

In [ ]:
p = {}
for index, row in df_p.iterrows():
    coffee = row["coffee"]
    profit = row["profit"]
    p[coffee] = profit
p

In [ ]:
a = {}
for index, row in df_a.iterrows():
    coffee = row["coffee"]
    bean = row["bean"]
    amount = row["amount"]
    a[bean, coffee] = amount
a

In [ ]:
b = {}
for index, row in df_b.iterrows():
    bean = row["bean"]
    stock = row["stock"]
    b[bean] = stock
b

In [ ]:
# 問題の設定: どのような問題（最大化・最小化、線形計画等）であるかを定義する
problem = pulp.LpProblem("CoffeeProduction", pulp.LpMaximize)

# 決定変数の設定
x = {}
for j in J:
    x[j] = pulp.LpVariable(f"x_{j}", cat="Continuous")

# 目的関数の設定
problem += sum([p[j] * x[j] for j in J])

# 制約条件の設定
for i in I:
    problem += sum([a[i, j] * x[j] for j in J]) <= b[i]
for j in J:
    problem += x[j] >= 0

problem